In [11]:
using CairoMakie, QuantEcon, LinearAlgebra, Optim, Dierckx

In [58]:
include("crra_code.jl")
include("ces_code.jl")

hpi_CES (generic function with 1 method)

In [78]:
pars = (;A = 5.0, # total factor productivity
    α = 0.33, # capital share
    δ = 0.08, # depreciation rate
    r = 0.00, # interest rate
    w = 1.0, # wage
    β = 0.96, # discount factor
    γ = 2.0, # risk aversion
    nz = 19, # number of grid points for z
    ρ = 0.9, # persistence of AR1
    μ = 0.0, # mean of AR1
    σ = 0.003, # std of AR1
    na = 101, # number of asset grid points
    ϕ = 0.0, # borrowing constraint
    θ = 4.0, # grid expansion factor
    toler = 4e-7, # tolerance
    maxiter = 1000, # maximum no. of iterations
    print_skip = 100, # how often to print
    how_iter = 75, # number of Howard iterations
    toler_prices = 1e-3, # tolerance
    maxiter_prices = 100, # maximum no. of iterations
    print_skip_prices = 10, # how often to print
    lb = 0.0, # lower bound of capital grid
    ub = 10000.0) # upper bound of capital grid

(A = 5.0, α = 0.33, δ = 0.08, r = 0.0, w = 1.0, β = 0.96, γ = 2.0, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.003, na = 101, ϕ = 0.0, θ = 4.0, toler = 4.0e-7, maxiter = 1000, print_skip = 100, how_iter = 75, toler_prices = 0.001, maxiter_prices = 100, print_skip_prices = 10, lb = 0.0, ub = 10000.0)

In [79]:
function invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
    (; na, nz) = pars
    for i in 1:na
        for j in 1:nz
            if policy[i,j] <+ Avals[1]
                M[i,j,1,:] = Π[j,:]
            elseif policy[i,j] >= Avals[end]
                M[i,j,end,:] = Π[j,:]
            else
                index = findfirst(x -> x > policy[i,j], Avals)
                π = (Avals[index] - policy[i,j]) / (Avals[index] - Avals[index-1])
                M[i,j,index-1,:] = π * Π[j,:]
                M[i,j,index,:] = (1-π) * Π[j,:]
            end
        end
    end
    O = reshape(M, n, n)
    for i in 1:na
        O[i,:] = O[i,:] / sum(O[i,:])
    end
    X = reduce(vcat, [I(n) - O', ones(1,n)])
    Y = [zeros(n);1]
    Inv = X \ Y
    Inv[Inv .< 0] .= 0
    Inv = Inv / sum(Inv)
    return Inv
end

invariant_distribution (generic function with 1 method)

In [80]:
function labour_supply(Π, Zvals, pars)
    (; nz) = pars
    L = zeros(nz)
    L2 = reduce(vcat, [I(nz) - Π', ones(1,nz)])
    L3 = [zeros(nz);1]
    L = L2 \ L3
    L[L .< 0] .= 0
    L = L / sum(L)
    L = L' * Zvals
    return L
end

labour_supply (generic function with 1 method)

In [81]:
function equilibrium(pars)
    (; na, nz, toler_prices, maxiter_prices, print_skip_prices, r, w, β, A, α, ϕ, δ) = pars
    n = na * nz
    M = zeros(na, nz, na, nz)
    O = zeros(n, n)
    X = zeros(n+1,n)
    Y = zeros(n+1)
    Inv = zeros(n)
    Π, Zvals = ar1(pars)
    Avals = exp_grid(pars)
    v_init = zeros(na,nz)
    policy = similar(v_init)
    wealth = zeros(na,nz)
    initial_distribution = zeros(n,1)
    L = labour_supply(Π, Zvals, pars)
    Ks, Kd = 1, 1
    iter = 0
    error = toler_prices + 1
    rnew = 1/β - 1
    if iter == 0
        println("Iterating on prices...")
    end
    while ((error > toler_prices) && (iter < maxiter_prices))
        println("//////////////////////")
        println("Price Iteration: $iter")
        r_iter = (r + rnew) / 2
        Kd = (A^α * L ^ (1-α)) / (r_iter + δ)^(1/(1-α))
        w_iter = (1-α) * A * (Kd/L)^α
        Φ = w_iter * (exp(minimum(Zvals))/r_iter)
        if ϕ > 0
            ϕ_iter = exp(minimum(Zvals))(Φ, ϕ)
            pars = (; pars..., ϕ = ϕ_iter, r = r_iter, w = w_iter)
        else
            pars = (; pars..., r = r_iter, w = w_iter)
        end
        v_new, policy = vfi(v_init, policy, Π, Zvals, Avals, pars)
        Invariant = invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
        G = reshape(policy .- ϕ, n, 1)
        Ks = dot(Invariant', G)
        diff = Ks - Kd
        error = abs(diff)
        if diff > 0
            rnew = r_iter
        else
            r = r_iter
        end
        if iter % print_skip_prices == 0
            println("%%%%%%%%%%%%%%%%%%")
            println("Iter = $iter, r = $r, w = $w, Ks = $Ks, Kd = $Kd, diff = $diff")
            println("%%%%%%%%%%%%%%%%%%")
        end
        v_init .= v_new
        iter += 1
    end
    r_final = r + rnew / 2
    w_final = (1-α) * A * (Kd/L)^α
    for i in 1:na
        for j in 1:nz
            wealth[i,j] = w_final * exp(Zvals[j]) + (1 + r_final) * Avals[i]
        end
    end
    return Ks, Kd, r_final, w_final, v_init, policy, Invariant, wealth
end

equilibrium (generic function with 1 method)

In [82]:
test = equilibrium(pars)

UndefVarError: UndefVarError: `Invariant` not defined